In [4]:
#import requisite library
import numpy as np
import pandas as pd
import plotly.express as px
from prophet import Prophet

In [6]:
#importing the dataset
df=pd.read_csv('/kaggle/input/temperature-change/Environment_Temperature_change_E_All_Data_NOFLAG.csv',encoding='latin-1')
df

,Area Code,Area,Months Code,Months,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,7001,January,7271,Temperature change,°C,0.777,0.062,2.744,...,3.601,1.179,-0.583,1.233,1.755,1.943,3.416,1.201,1.996,2.951
1,2,Afghanistan,7001,January,6078,Standard Deviation,°C,1.950,1.950,1.950,...,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950
2,2,Afghanistan,7002,February,7271,Temperature change,°C,-1.743,2.465,3.919,...,1.212,0.321,-3.201,1.494,-3.187,2.699,2.251,-0.323,2.705,0.086
3,2,Afghanistan,7002,February,6078,Standard Deviation,°C,2.597,2.597,2.597,...,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597
4,2,Afghanistan,7003,March,7271,Temperature change,°C,0.516,1.336,0.403,...,3.390,0.748,-0.527,2.246,-0.076,-0.497,2.296,0.834,4.418,0.234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9651,5873,OECD,7018,JunJulAug,6078,Standard Deviation,°C,0.247,0.247,0.247,...,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247
9652,5873,OECD,7019,SepOctNov,7271,Temperature change,°C,0.036,0.461,0.665,...,0.958,1.106,0.885,1.041,0.999,1.670,1.535,1.194,0.581,1.233
9653,5873,OECD,7019,SepOctNov,6078,Standard Deviation,°C,0.378,0.378,0.378,...,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378
9654,5873,OECD,7020,Meteorological year,7271,Temperature change,°C,0.165,-0.009,0.134,...,1.246,0.805,1.274,0.991,0.811,1.282,1.850,1.349,1.088,1.297


In [7]:
#checking for missing values in the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9656 entries, 0 to 9655
Data columns (total 66 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Area Code     9656 non-null   int64  
 1   Area          9656 non-null   object 
 2   Months Code   9656 non-null   int64  
 3   Months        9656 non-null   object 
 4   Element Code  9656 non-null   int64  
 5   Element       9656 non-null   object 
 6   Unit          9656 non-null   object 
 7   Y1961         8287 non-null   float64
 8   Y1962         8322 non-null   float64
 9   Y1963         8294 non-null   float64
 10  Y1964         8252 non-null   float64
 11  Y1965         8281 non-null   float64
 12  Y1966         8364 non-null   float64
 13  Y1967         8347 non-null   float64
 14  Y1968         8345 non-null   float64
 15  Y1969         8326 non-null   float64
 16  Y1970         8308 non-null   float64
 17  Y1971         8303 non-null   float64
 18  Y1972         8323 non-null 

In [24]:
#creating preprocessing function 
def preprocessing_input(df):
    df=df.copy()


    #removing the Standard Deviation examples from the dataset


    df=df.query("Element=='Temperature change'")

    #grouping the countries and take their mean

    df=df.groupby('Area').mean(numeric_only=True)



    #dropping the first three columns

    df=df.loc[:,'Y1961':]

    df=pd.DataFrame(df.mean()).reset_index(drop=False)
    
    #Rename columns

    df.columns=['ds','y']

    #Fixing year columns

    df['ds']=df['ds'].apply(lambda x:x[1:]).astype(int)


    
    return df

In [29]:
time_series=preprocessing_input(df)
time_series

,ds,y
0,1961,0.143032
1,1962,-0.028398
2,1963,-0.026297
3,1964,-0.122865
4,1965,-0.224154
5,1966,0.095070
6,1967,-0.131975
7,1968,-0.167841
8,1969,0.105694
9,1970,0.072189


In [31]:
fig=px.line(time_series,x='ds',y='y',labels={'ds':'Year','y':'Change in temperature (C)'},
           title='Average Global Temperature Change over time')
fig.show()

**In-Sample forecast**

In [34]:
time_train=time_series.iloc[0:44,:].copy()
time_test=time_series.iloc[44:].copy()

In [35]:
time_train

,ds,y
0,1961,0.143032
1,1962,-0.028398
2,1963,-0.026297
3,1964,-0.122865
4,1965,-0.224154
5,1966,0.095070
6,1967,-0.131975
7,1968,-0.167841
8,1969,0.105694
9,1970,0.072189


In [36]:
time_test

,ds,y
44,2005,0.886901
45,2006,0.910877
46,2007,1.004826
47,2008,0.813313
48,2009,0.943937
49,2010,1.080097
50,2011,0.863045
51,2012,0.901637
52,2013,0.977131
53,2014,1.131417


In [37]:
in_model=Prophet()
in_model.fit(time_train)

05:18:02 - cmdstanpy - INFO - Chain [1] start processing
05:18:02 - cmdstanpy - INFO - Chain [1] done processing


In [43]:
in_forecast=in_model.predict(time_test).loc[:,['ds','yhat']]

In [45]:
in_forecast['ds']=in_forecast['ds'].apply(lambda x:x.year)

In [46]:
in_forecast

,ds,yhat
0,2005,0.811440
1,2006,0.809407
2,2007,0.796937
3,2008,0.774072
4,2009,0.901347
5,2010,0.899313
6,2011,0.886843
7,2012,0.863978
8,2013,0.991253
9,2014,0.989220


In [50]:
in_result_df=time_series.merge(in_forecast,on='ds',how='left')

In [51]:
in_result_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning:

invalid value encountered in greater

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in less

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in greater



,ds,y,yhat
0,1961,0.143032,NaN
1,1962,-0.028398,NaN
2,1963,-0.026297,NaN
3,1964,-0.122865,NaN
4,1965,-0.224154,NaN
5,1966,0.095070,NaN
6,1967,-0.131975,NaN
7,1968,-0.167841,NaN
8,1969,0.105694,NaN
9,1970,0.072189,NaN


In [54]:
fig=px.line(in_result_df,x='ds',y=['y','yhat'],color_discrete_sequence=['black','red'],labels={'ds':'Year'},
           title='Average Global Temperature Change over time')
fig.show()

# Out of Sample Forecast

In [56]:
future_df=pd.DataFrame(np.arange(2020,2046),columns=['ds'])
future_df

,ds
0,2020
1,2021
2,2022
3,2023
4,2024
5,2025
6,2026
7,2027
8,2028
9,2029


In [57]:
out_model=Prophet()

out_model.fit(time_series)

05:31:48 - cmdstanpy - INFO - Chain [1] start processing
05:31:48 - cmdstanpy - INFO - Chain [1] done processing


In [60]:
out_forecast=out_model.predict(future_df).loc[:,['ds','yhat']]
out_forecast['ds']=out_forecast['ds'].apply(lambda x:x.year)
out_forecast

,ds,yhat
0,2020,1.216595
1,2021,1.328158
2,2022,1.354660
3,2023,1.354065
4,2024,1.326409
5,2025,1.437972
6,2026,1.464474
7,2027,1.463879
8,2028,1.436223
9,2029,1.547786


In [61]:
out_result_df=pd.concat([time_series,out_forecast],axis=0)


In [62]:
out_result_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning:

invalid value encountered in greater

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in less

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in greater



,ds,y,yhat
0,1961,0.143032,NaN
1,1962,-0.028398,NaN
2,1963,-0.026297,NaN
3,1964,-0.122865,NaN
4,1965,-0.224154,NaN
...,...,...,...
21,2041,NaN,1.877229
22,2042,NaN,1.903730
23,2043,NaN,1.903136
24,2044,NaN,1.875480


In [63]:
fig=px.line(out_result_df,x='ds',y=['y','yhat'],color_discrete_sequence=['black','red'],labels={'ds':'Year'},
           title='Average Global Temperature Change over time')
fig.show()